# Ultimate Processing Performance

## Multi-stage data processing pipeline

В отличие от TensorFlow и PyTorch, MindSpore использует многоступенчатый параллельный конвейер для построения пайплайна обработки данных, что позволяет более детально планировать использование вычислительных ресурсов. Rаждая операция с набором данных содержит выходной коннектор, представляющий собой буферную очередь, сохраняющую порядок, состоящую из набора блокирующих очередей и счётчиков. Каждая операция с набором данных берёт кэшированные данные из коннектора предыдущей операции для обработки, а затем помещает этот кэш в свой выходной коннектор, и так далее.

Преимущества подхода: 
1) Операции загрузки набора данных, преобразования (map), пакетирования (batch) и другие управляются механизмом планирования задач. Задачи для каждой операции независимы друг от друга, а контексты связаны через коннекторы.
2) Каждая операция может быть реализована с тонкой настройкой многопоточной или многопроцессорной параллельной обработки. Фреймворк данных предоставляет интерфейсы для пользователей, позволяя регулировать количество потоков для каждой операции и управлять многопроцессорной обработкой, что даёт пользователям возможность гибко контролировать скорость обработки на каждом узле и, таким образом, добиваться оптимальной производительности всего пайплайна обработки данных.
3) Пользователи могут задавать размер коннектора, что в определённой степени позволяет эффективно контролировать использование памяти и адаптироваться к различным требованиям сетей по производительности обработки данных.

С этим механизмом обработки данных важным аспектом для обеспечения точности обучения является сохранение порядка вывода данных. Сохранение порядка означает, что пайплайн обработки данных выводит данные в том же порядке, в котором они поступали до обработки. MindSpore использует алгоритм циклического распределения (round robin), чтобы обеспечить упорядоченность данных при многопоточной обработке.

**Алгоритм round robin:** Когда несколько потоков одновременно обрабатывают данные (например, 4 потока выполняют операцию "map"), данные в очередях должны забираться и обрабатываться так, чтобы не нарушить порядок. Для этого каждый поток использует два счетчика:
1) `expect_consumer_` следит за количеством потребителей данных, и он увеличивается при каждом извлечении данных.
2) `pop_from_` контролирует, из какой именно внутренней очереди данные будут извлекаться в следующий раз.

Оба счетчика работают по принципу цикличности: когда поток доходит до конца одной очереди, он возвращается в начало, и так происходит распределение задач между потоками без нарушения порядка данных.

## Data processing and network computing pipeline

Конвейер обработки данных непрерывно обрабатывает данные и отправляет обработанные данные в кэш на стороне устройства, и после выполнения одного шага данные следующего шага считываются непосредственно из кэша устройства.

Сам pipeline состоит из следующих компонент:
1) **Обработка данных:** обработка набора данных в необходимый для сети формат и передача его в очередь отправки для эффективной обработки данных.
2) **Очередь отправки:** поддержание очередей данных для обеспечения независимости процессов обработки данных и вычислений сети, действуя как мостик.
3) **Вычисления в сети:** получение данных из очереди отправки для итеративного обучения.

Каждая из указанных выше компонент имеет свою роль, независимую от других, для формирования полного процесса обучения. Поэтому, пока очередь данных не пуста, обучение модели не будет блокироваться из-за ожидания обучающих данных



## Cache Technology

Когда размер набора данных слишком велик, чтобы полностью загрузить его в кэш в памяти, некоторые данные, используемые для обучения, необходимо считывать с диска, и могут возникнуть проблемы с вводом-выводом, поэтому увеличение коэффициента использования кэша в каждую эпоху особенно важно. Традиционное управление кэшем использует стратегию LRU, которая не учитывает характеристики чтения данных глубокого обучения, т.е. данные считываются повторно между разными эпохами, в то время как в одну и ту же эпоху они считываются случайным образом. Каждый фрагмент данных имеет одинаковую вероятность быть прочитанным, поэтому не имеет значения, какой фрагмент данных кэширован. Вместо этого более важно, чтобы кэшированные данные не были выгружены перед их использованием. Для этой функции мы используем простой и эффективный алгоритм кэширования, т.е. после кэширования данных они не выгружаются из кэша.

Во время оптимизации графа данных MindSpore автоматически генерирует операторы кэширования на основе структуры конвейера, кэшируя как исходный набор данных, так и результаты после обработки дополнительных данных.


**Замечание: LRU (Least Recently Used)** — это стратегия управления кэшем, которая при переполнении памяти удаляет те данные, которые использовались наименее недавно. Основная идея LRU заключается в том, что если данные долгое время не использовались, то с наибольшей вероятностью они не будут востребованы в ближайшее время. Поэтому такие данные заменяются новыми.

# Flexible Customization Capabilities

Пользователи часто имеют различные потребности в обработке данных, и логика обработки, которая не закреплена в платформе, должна поддерживаться открытыми возможностями настройки. В результате MindSpore предоставляет гибкие методы загрузки наборов данных, широкие возможности операций обработки данных и такие механизмы, как автоматическое дополнение данных, динамическая форма и обратный вызов обработки данных, которые разработчики могут использовать в различных сценариях.

Более подробно: https://www.mindspore.cn/docs/en/r2.3.1/design/data_engine.html
